# Week 4: Modern Classification Networks

In [ ]:
## Functions 

In [1]:
#!/bin/env python

# Example of large scale dataset processing in Tensorflow.
# Processes the ImageNet dataset into a one-hot classificaiton
# dataset.
#
# ImageNet is a mixture of images, with 1000 labeled classes.
# Each image can have one or more class objects.
# The annotations for each image includes class ID and bounding
# box dimensions. The functions below use these bounding boxes
# to chop up the original images to create single images
# corresponding to single class labels. This simplifies the
# network needed to label the data, but effects the final
# network accuracy.
#
# questions? Taylor Childers, jchilders@anl.gov

import os,glob
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import xml.etree.ElementTree as ET
import horovod.tensorflow as hvd
hvd.init()

# these are initialized in the get_datasets function and used later
labels_hash = None
crop_size = None


def get_datasets(config):
   # these global variables will be initizlized
    global labels_hash,crop_size

   # set the crop size of the output images, e.g. [256,256]
    crop_size = tf.constant(config['data']['crop_image_size'])
   # these are paths to text files containing a list, one entry per line,
   # of all the training JPEGs and testing JPEGs
   # it's assumed the full path to the JPEGs is like this:
   # /.../ILSVRC/Data/CLS-LOC/train/n02437312/n02437312_8688.JPEG
   # because the class label comes from the last folder text.
    train_filelist = config['data']['train_filelist']
    test_filelist = config['data']['test_filelist']

    assert os.path.exists(train_filelist)
    assert os.path.exists(test_filelist)

   # this function uses that class label from the filename path
   # and builds a map from the string labels like the above "n02537312"
   # to a unique integer value 0-999. This is more suitable for
   # network classifciation than a string.
    labels_hash = get_label_tables(config, train_filelist)

   # this function creates the tf.dataset.Dataset objects for each list
   # of input JPEGs.
    train_ds = build_dataset_from_filelist(config,train_filelist)
    valid_ds = build_dataset_from_filelist(config,test_filelist)

    return train_ds,valid_ds


## Create a hash table for labels from string to int 
def get_label_tables(config, train_filelist):

   # get the first filename
    with open(train_filelist) as file:
        filepath = file.readline().strip()

   # parse the filename to extract the "n02537312" string
   # from the full path which is assumed to be similar to this
   # /.../ILSVRC/Data/CLS-LOC/train/n02437312/n02437312_8688.JPEG
   # and convert that string to a unique value from 0-999

   # this extracts the path up to: /.../ILSVRC/Data/CLS-LOC/train/
    label_path = '/'.join(filepath.split('/')[:-2])
   # this globs for all the directories like "n02537312" to get 
   # list of the string labels
    labels = glob.glob(label_path + os.path.sep + '*')
    if config['hvd'].rank() == 0:
        print(f'num labels: {len(labels)}')
   # this removes the leading path from the label directories
    labels = [os.path.basename(i) for i in labels]
   # create a list of integers as long as the number of labels
    hash_values = tf.range(len(labels))
   # convert python list of strings to a tensorflow vector
    hash_keys = tf.constant(labels, dtype=tf.string)
   # build a key-value lookup using Tensorflow tools
    labels_hash_init = tf.lookup.KeyValueTensorInitializer(hash_keys, hash_values)
   # build a lookup table based on those key-value pairs (returns -1 for undefined keys)
    labels_hash = tf.lookup.StaticHashTable(labels_hash_init, -1)

    return labels_hash


# take a config dictionary and a path to a filelist
# return a tf.dataset.Dataset object that will iterate over the JPEGs in filelist
def build_dataset_from_filelist(config,filelist_filename):
   
    if config['hvd'].rank() == 0:
        print(f'build dataset {filelist_filename}')

   
    dc = config['data']

   # if running horovod(MPI) need to shard the dataset based on rank
    numranks = 1
   
    if config['hvd']:
        numranks = config['hvd'].size()

   # loading full filelist
    filelist = []
   
    with open(filelist_filename) as file:
        for line in file:
            filelist.append(line.strip())

   # provide user with estimated batches in epoch
    batches_per_rank = int(len(filelist) / dc['batch_size'] / numranks)
    if config['hvd'].rank() == 0:
        print(f'input filelist contains {len(filelist)} files, estimated batches per rank {batches_per_rank}')
   
   # convert python list to tensorflow vector object
    filelist = tf.data.Dataset.from_tensor_slices(filelist)

   # if using horovod (MPI) shard the data based on total ranks (size) and rank
    if config['hvd']:
        filelist = filelist.shard(config['hvd'].size(), config['hvd'].rank())
   
   # shuffle the data, set shuffle buffer (needs to be large), and reshuffle after each epoch
    filelist = filelist.shuffle(dc['shuffle_buffer'],reshuffle_each_iteration=dc['reshuffle_each_iteration'])

   # run 'load_image_label_bb' on each input image file, process multiple files in parallel
   # this function opens the JPEG, converts it to a tensorflow vector and gets the truth class label
    ds = filelist.map(load_image_label_bb,
                     num_parallel_calls=tf.data.experimental.AUTOTUNE)
                     
   # unbatch called because some JPEGs result in more than 1 image returned
    ds = ds.apply(tf.data.Dataset.unbatch)

   # batch the data
    ds = ds.batch(dc['batch_size'])

   # setup a pipeline that pre-fetches images before they are needed (keeps CPU busy)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)  

    return ds


# this function parses the image path, uses the label hash to convert the string
# label in the path to a numerical label, decodes the input JPEG, and returns
# the input image and label
def load_image_label_bb(image_path):
   
   # for each JPEG, there is an Annotation file that contains a list of
   # classes contained in the image and a bounding box for each object.
   # However, some images contain a single class, in which case the
   # dataset contains no annotation file which is annoying, but...
   # Images with multiple objects per file are always the same class.
    label = tf.strings.split(image_path, os.path.sep)[-2]
    annot_path = tf.strings.regex_replace(image_path,'Data','Annotations')
    annot_path = tf.strings.regex_replace(annot_path,'JPEG','xml')

   # open the annotation file and retrieve the bounding boxes and indices
    bounding_boxes,box_indices = tf.py_function(get_bounding_boxes,[annot_path],[tf.float32,tf.int32])

   # open the JPEG
    img = tf.io.read_file(image_path)
   # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
   # add batching index [batch,width,height,channel]
    img = tf.expand_dims(img,0)

   # create individual images based on bounding boxes
    imgs = tf.image.crop_and_resize(img,bounding_boxes,box_indices,crop_size)

   # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    imgs = tf.image.convert_image_dtype(imgs, tf.float16)
   # resize the image to the desired size. networks don't like variable sized arrays.
    imgs = tf.image.resize(imgs, crop_size)
   # convert string label to numerical label
    label = labels_hash.lookup(label)
   # duplicate labels to match the number of images created from bounding boxes
    labels = tf.fill([tf.shape(imgs)[0]],label)
   # return images and labels
    return imgs, labels


# this function opens the annotation XML file and parses the contents
# the contents include a list of objects in the JPEG, a label and
# bounding box for each object
def get_bounding_boxes(filename):
    filename = bytes.decode(filename.numpy())
    try:
        tree = ET.parse(filename)
        root = tree.getroot()

        img_size = root.find('size')
        img_width = int(img_size.find('width').text)
        img_height = int(img_size.find('height').text)
      # img_depth = int(img_size.find('depth').text)

        objs = root.findall('object')
        bndbxs = []
      # label = None
        for object in objs:
         # label = object.find('name').text
            bndbox = object.find('bndbox')
            bndbxs.append([
                float(bndbox.find('ymin').text) / (img_height - 1),
                float(bndbox.find('xmin').text) / (img_width - 1),
                float(bndbox.find('ymax').text) / (img_height - 1),
                float(bndbox.find('xmax').text) / (img_width - 1)
             ])
    except FileNotFoundError:
        bndbxs = [[0,0,1,1]]

    return np.asarray(bndbxs,float),np.zeros(len(bndbxs))



if __name__ == '__main__':
   # parse command line
    import argparse,json,time
    parser = argparse.ArgumentParser(description='test this')
    parser.add_argument('-c', '--config', dest='config_filename',
                       help='configuration filename in json format',
                       required=True)
    parser.add_argument('-l','--logdir', dest='logdir',
                       help='log output directory',default='logdir')
    parser.add_argument('-n','--nsteps', dest='nsteps',
                       help='number of steps to run',default=10,type=int)
    parser.add_argument('--interop',type=int,help='set Tensorflow "inter_op_parallelism_threads" session config varaible ',default=None)
    parser.add_argument('--intraop',type=int,help='set Tensorflow "intra_op_parallelism_threads" session config varaible ',default=None)

    args = parser.parse_args()

    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    if gpus:
        tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

    print("GPUs Available: %s" % tf.config.get_visible_devices('GPU'))

   # parse config file
    config = json.load(open(args.config_filename))
    config['hvd'] = hvd
   
   # define some parallel processing sizes
    if args.interop is not None:
        tf.config.threading.set_inter_op_parallelism_threads(args.interop)
    if args.intraop is not None:
        tf.config.threading.set_intra_op_parallelism_threads(args.intraop)
   
   # use the tensorflow profiler here
    if hvd.rank() == 0:
        tf.profiler.experimental.start(args.logdir)
   # call function to build dataset objects
   # both of the returned objects are tf.dataset.Dataset objects
    trainds, testds = get_datasets(config)
   # can iterate over a dataset object
    trainds = iter(trainds)
    start = time.time()
    for i in range(args.nsteps):
      # profile data pipeline
        with tf.profiler.experimental.Trace('train_%02d' % i, step_num=i, _r=1):
            inputs,labels = next(trainds)
      
      # print('batch_number = %s input shape = %s    labels shape = %s' %(i,inputs.shape,labels.shape))
      # print('batch_number = %s labels = %s' %(i,labels))
   # measure performance in images per second
    duration = time.time() - start
    if hvd.rank() == 0:
        tf.profiler.experimental.stop()
    images = config['data']['batch_size'] * args.nsteps
    if hvd.rank() == 0:
        print('imgs/sec = %5.2f' % ((images/duration)*hvd.size()))


--------------------------------------------------------------------------

  Local host:   thetagpu06
  Local device: mlx5_0
--------------------------------------------------------------------------
usage: ipykernel_launcher.py [-h] -c CONFIG_FILENAME [-l LOGDIR] [-n NSTEPS]
                             [--interop INTEROP] [--intraop INTRAOP]
ipykernel_launcher.py: error: the following arguments are required: -c/--config


SystemExit: 2

/lus/theta-fs0/software/thetagpu/conda/2022-07-01/mconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Run CNN on GPU

In [4]:
import sys, os
import time
import pandas as pd
# This limits the amount of memory used:
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=2"
global data_schedule 
data_schedule=[]
import tensorflow as tf



#########################################################################
# Here's the Residual layer from the first half again:
#########################################################################
class ResidualLayer(tf.keras.Model):

    def __init__(self, n_filters):
        # tf.keras.Model.__init__(self)
        super(ResidualLayer, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(
            filters     = n_filters,
            kernel_size = (3,3),
            padding     = "same"
        )

        self.norm1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(
            filters     = n_filters,
            kernel_size = (3,3),
            padding     = "same"
        )

        self.norm2 = tf.keras.layers.BatchNormalization()

    def call(self, inputs):

        x = inputs

        output1 = self.norm1(self.conv1(inputs))

        output1 = tf.keras.activations.relu(output1)

        output2 = self.norm2(self.conv2(output1))

        return tf.keras.activations.relu(output2 + x)

#########################################################################
# Here's layer that does a spatial downsampling:
#########################################################################
class ResidualDownsample(tf.keras.Model):

    def __init__(self, n_filters):
        # tf.keras.Model.__init__(self)
        super(ResidualDownsample, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(
            filters     = n_filters,
            kernel_size = (3,3),
            padding     = "same",
            strides     = (2,2)
        )

        self.identity = tf.keras.layers.Conv2D(
            filters     = n_filters,
            kernel_size = (1,1),
            strides     = (2,2),
            padding     = "same"
        )

        self.norm1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(
            filters     = n_filters,
            kernel_size = (3,3),
            padding     = "same"
        )

        self.norm2 = tf.keras.layers.BatchNormalization()

    @tf.function
    def call(self, inputs):

        x = self.identity(inputs)
        output1 = self.norm1(self.conv1(inputs))
        output1 = tf.keras.activations.relu(output1)

        output2 = self.norm2(self.conv2(output1))

        return tf.keras.activations.relu(output2 + x)


#########################################################################
# Armed with that, let's build ResNet (this particular one is called ResNet34)
#########################################################################

class ResNet34(tf.keras.Model):

    def __init__(self):
        super(ResNet34, self).__init__()

        self.conv_init = tf.keras.Sequential([
            tf.keras.layers.Conv2D(
                filters     = 64,
                kernel_size = (7,7),
                strides     = (2,2),
                padding     = "same",
                use_bias    = False
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding="same")

        ])

        self.residual_series_1 = tf.keras.Sequential([
            ResidualLayer(64),
            ResidualLayer(64),
            ResidualLayer(64),
        ])

        # Increase the number of filters:
        self.downsample_1 = ResidualDownsample(128)

        self.residual_series_2 = tf.keras.Sequential([
            ResidualLayer(128),
            ResidualLayer(128),
            ResidualLayer(128),
        ])

        # Increase the number of filters:
        self.downsample_2 = ResidualDownsample(256)

        self.residual_series_3 = tf.keras.Sequential([
            ResidualLayer(256),
            ResidualLayer(256),
            ResidualLayer(256),
            ResidualLayer(256),
            ResidualLayer(256),
        ])

        # Increase the number of filters:
        self.downsample_3 = ResidualDownsample(512)


        self.residual_series_4 = tf.keras.Sequential([
            ResidualLayer(512),
            ResidualLayer(512),
        ])

        self.final_pool = tf.keras.layers.AveragePooling2D(
            pool_size=(8,8)
        )

        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.layers.Dense(1000)

    @tf.function
    def call(self, inputs):

        x = self.conv_init(inputs)

        x = self.residual_series_1(x)


        x = self.downsample_1(x)


        x = self.residual_series_2(x)

        x = self.downsample_2(x)

        x = self.residual_series_3(x)

        x = self.downsample_3(x)


        x = self.residual_series_4(x)


        x = self.final_pool(x)

        x = self.flatten(x)

        logits = self.classifier(x)

        return logits




@tf.function
def calculate_accuracy(logits, labels):
    # We calculate top1 accuracy only here:
    selected_class = tf.argmax(logits, axis=1)

    correct = tf.cast(selected_class, tf.float32) == tf.cast(labels, tf.float32)

    return tf.reduce_mean(tf.cast(correct, tf.float32))


@tf.function
def calculate_loss(logits, labels):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels, logits)
    return tf.reduce_mean(loss)

@tf.function
def training_step(network, optimizer, images, labels):
    with tf.GradientTape() as tape:
        logits = network(images)
        loss = calculate_loss(logits, labels)

    gradients = tape.gradient(loss, network.trainable_variables)

    optimizer.apply_gradients(zip(gradients, network.trainable_variables))

    accuracy = calculate_accuracy(logits, labels)

    return loss, accuracy

def train_epoch(i_epoch, step_in_epoch, train_ds, val_ds, network, optimizer, BATCH_SIZE, checkpoint):
    # Here is our training loop!

    steps_per_epoch = int(1281167 / BATCH_SIZE)
    steps_validation = int(50000 / BATCH_SIZE)

    start = time.time()
    for train_images, train_labels in train_ds.take(steps_per_epoch):
        if step_in_epoch > steps_per_epoch: break
        else: step_in_epoch.assign_add(1)

        # Peform the training step for this batch
        loss, acc = training_step(network, optimizer, train_images, train_labels)
        end = time.time()
        images_per_second = BATCH_SIZE / (end - start)
        print(f"Finished step {step_in_epoch.numpy()} of {steps_per_epoch} in epoch {i_epoch.numpy()},loss={loss:.3f}, acc={acc:.3f} ({images_per_second:.3f} img/s).")
        data_schedule.append({'epoch':i_epoch.numpy(),'iteration/step':steps_per_epoch,
                              'loss':loss.numpy(),
                             'accuracy':acc.numpy()})
        start = time.time()

    # Save the network after every epoch:
    checkpoint.save("resnet34/model")

    # Compute the validation accuracy:
    mean_accuracy = None
    for val_images, val_labels in val_ds.take(steps_validation):
        logits = network(val_images)
        accuracy = calculate_accuracy(logits, val_labels)
        if mean_accuracy is None:
            mean_accuracy = accuracy
        else:
            mean_accuracy += accuracy

    mean_accuracy /= steps_validation

    print(f"Validation accuracy after epoch {i_epoch.numpy()}: {mean_accuracy:.4f}.")



def prepare_data_loader(BATCH_SIZE):


    tf.config.threading.set_inter_op_parallelism_threads(8)
    tf.config.threading.set_intra_op_parallelism_threads(8)

    print("Parameters set, preparing dataloading")
    #########################################################################
    # Here's the part where we load datasets:
    import json


    # What's in this function?  Tune in next week ...
    from ilsvrc_dataset import get_datasets


    class FakeHvd:

        def size(self): return 1

        def rank(self): return 0


    with open("ilsvrc.json", 'r') as f:
        config = json.load(f)

    print(json.dumps(config, indent=4))


    config['hvd'] = FakeHvd()
    config['data']['batch_size'] = BATCH_SIZE

    train_ds, val_ds = get_datasets(config)
    print("Datasets ready, creating network.")
    #########################################################################

    return train_ds, val_ds


def main():
    #########################################################################
    # Here's some configuration:
    #########################################################################
    BATCH_SIZE = 256
    N_EPOCHS = 4

    train_ds, val_ds = prepare_data_loader(BATCH_SIZE)


    example_images, example_labels = next(iter(train_ds.take(1)))


    print("Initial Image size: ", example_images.shape)
    network = ResNet34()

    output = network(example_images)
    print("output shape:", output.shape)

    print(network.summary())




    epoch = tf.Variable(initial_value=tf.constant(0, dtype=tf.dtypes.int64), name='epoch')
    step_in_epoch = tf.Variable(
        initial_value=tf.constant(0, dtype=tf.dtypes.int64),
        name='step_in_epoch')


    # We need an optimizer.  Let's use Adam:
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    checkpoint = tf.train.Checkpoint(
        network       = network,
        optimizer     = optimizer,
        epoch         = epoch,
        step_in_epoch = step_in_epoch)

    # Restore the model, if possible:
    latest_checkpoint = tf.train.latest_checkpoint("resnet34/")
    if latest_checkpoint:
        checkpoint.restore(latest_checkpoint)

    while epoch < N_EPOCHS:
        train_epoch(epoch, step_in_epoch, train_ds, val_ds, network, optimizer, BATCH_SIZE, checkpoint)
        epoch.assign_add(1)
        step_in_epoch.assign(0)
    data_frame=pd.DataFrame(data_schedule)
    data_frame.to_csv("loss_accuracy_data.csv")

if __name__ == "__main__":
    main()


Parameters set, preparing dataloading
{
    "data": {
        "handler": "ilsvrc_dataset",
        "batch_size": 128,
        "train_filelist": "/lus/grand/projects/ALCFAITP/ilsvrc_train_filelist.txt",
        "test_filelist": "/lus/grand/projects/ALCFAITP/ilsvrc_val_filelist.txt",
        "shuffle_buffer": 200000,
        "reshuffle_each_iteration": true,
        "num_parallel_readers": 8,
        "prefectch_buffer_size": 10,
        "crop_image_size": [
            256,
            256
        ],
        "num_classes": 1000,
        "num_channels": 3
    }
}
num labels: 1000
build dataset /lus/grand/projects/ALCFAITP/ilsvrc_train_filelist.txt
input filelist contains 1281167 files, estimated batches per rank 5004
build dataset /lus/grand/projects/ALCFAITP/ilsvrc_val_filelist.txt
input filelist contains 50000 files, estimated batches per rank 195
Datasets ready, creating network.
Initial Image size:  (256, 256, 256, 3)
output shape: (256, 1000)
Model: "res_net34_2"
____________________

Finished step 69 of 5004 in epoch 3,loss=1.907, acc=0.523 (646.361 img/s).
Finished step 70 of 5004 in epoch 3,loss=1.635, acc=0.586 (566.073 img/s).
Finished step 71 of 5004 in epoch 3,loss=1.653, acc=0.613 (533.978 img/s).
Finished step 72 of 5004 in epoch 3,loss=1.713, acc=0.559 (606.110 img/s).
Finished step 73 of 5004 in epoch 3,loss=1.460, acc=0.613 (584.093 img/s).
Finished step 74 of 5004 in epoch 3,loss=1.592, acc=0.590 (650.946 img/s).
Finished step 75 of 5004 in epoch 3,loss=1.610, acc=0.574 (626.431 img/s).
Finished step 76 of 5004 in epoch 3,loss=1.545, acc=0.578 (600.927 img/s).
Finished step 77 of 5004 in epoch 3,loss=1.450, acc=0.613 (691.202 img/s).
Finished step 78 of 5004 in epoch 3,loss=1.394, acc=0.656 (591.771 img/s).
Finished step 79 of 5004 in epoch 3,loss=1.764, acc=0.559 (63.227 img/s).
Finished step 80 of 5004 in epoch 3,loss=1.425, acc=0.609 (600.960 img/s).
Finished step 81 of 5004 in epoch 3,loss=1.493, acc=0.609 (668.096 img/s).
Finished step 82 of 5004 i

Finished step 178 of 5004 in epoch 3,loss=1.607, acc=0.551 (650.187 img/s).
Finished step 179 of 5004 in epoch 3,loss=1.533, acc=0.598 (618.460 img/s).
Finished step 180 of 5004 in epoch 3,loss=1.596, acc=0.602 (588.997 img/s).
Finished step 181 of 5004 in epoch 3,loss=1.511, acc=0.594 (634.470 img/s).
Finished step 182 of 5004 in epoch 3,loss=1.522, acc=0.594 (608.507 img/s).
Finished step 183 of 5004 in epoch 3,loss=1.531, acc=0.613 (633.899 img/s).
Finished step 184 of 5004 in epoch 3,loss=1.579, acc=0.590 (632.674 img/s).
Finished step 185 of 5004 in epoch 3,loss=1.798, acc=0.535 (713.508 img/s).
Finished step 186 of 5004 in epoch 3,loss=1.410, acc=0.633 (683.822 img/s).
Finished step 187 of 5004 in epoch 3,loss=1.634, acc=0.590 (584.718 img/s).
Finished step 188 of 5004 in epoch 3,loss=1.680, acc=0.566 (516.484 img/s).
Finished step 189 of 5004 in epoch 3,loss=1.641, acc=0.578 (647.810 img/s).
Finished step 190 of 5004 in epoch 3,loss=1.763, acc=0.570 (615.371 img/s).
Finished ste

Finished step 286 of 5004 in epoch 3,loss=1.642, acc=0.582 (604.846 img/s).
Finished step 287 of 5004 in epoch 3,loss=1.387, acc=0.637 (611.012 img/s).
Finished step 288 of 5004 in epoch 3,loss=1.615, acc=0.586 (609.330 img/s).
Finished step 289 of 5004 in epoch 3,loss=1.484, acc=0.625 (640.687 img/s).
Finished step 290 of 5004 in epoch 3,loss=1.772, acc=0.578 (586.990 img/s).
Finished step 291 of 5004 in epoch 3,loss=1.473, acc=0.641 (602.537 img/s).
Finished step 292 of 5004 in epoch 3,loss=1.755, acc=0.547 (628.847 img/s).
Finished step 293 of 5004 in epoch 3,loss=1.645, acc=0.547 (671.425 img/s).
Finished step 294 of 5004 in epoch 3,loss=1.771, acc=0.559 (593.195 img/s).
Finished step 295 of 5004 in epoch 3,loss=1.606, acc=0.605 (673.917 img/s).
Finished step 296 of 5004 in epoch 3,loss=1.531, acc=0.605 (568.398 img/s).
Finished step 297 of 5004 in epoch 3,loss=1.550, acc=0.609 (699.566 img/s).
Finished step 298 of 5004 in epoch 3,loss=1.593, acc=0.574 (692.596 img/s).
Finished ste

Finished step 394 of 5004 in epoch 3,loss=1.675, acc=0.578 (651.491 img/s).
Finished step 395 of 5004 in epoch 3,loss=1.780, acc=0.543 (649.698 img/s).
Finished step 396 of 5004 in epoch 3,loss=1.634, acc=0.590 (598.340 img/s).
Finished step 397 of 5004 in epoch 3,loss=1.852, acc=0.527 (666.143 img/s).
Finished step 398 of 5004 in epoch 3,loss=1.708, acc=0.574 (692.946 img/s).
Finished step 399 of 5004 in epoch 3,loss=1.493, acc=0.625 (646.783 img/s).
Finished step 400 of 5004 in epoch 3,loss=1.780, acc=0.531 (685.307 img/s).
Finished step 401 of 5004 in epoch 3,loss=1.805, acc=0.566 (680.413 img/s).
Finished step 402 of 5004 in epoch 3,loss=1.577, acc=0.605 (566.328 img/s).
Finished step 403 of 5004 in epoch 3,loss=1.655, acc=0.613 (696.047 img/s).
Finished step 404 of 5004 in epoch 3,loss=1.744, acc=0.562 (611.180 img/s).
Finished step 405 of 5004 in epoch 3,loss=1.744, acc=0.551 (603.370 img/s).
Finished step 406 of 5004 in epoch 3,loss=1.736, acc=0.578 (698.201 img/s).
Finished ste

...

Finished step 4573 of 5004 in epoch 3,loss=1.940, acc=0.559 (696.479 img/s).
Finished step 4574 of 5004 in epoch 3,loss=1.954, acc=0.539 (746.764 img/s).
Finished step 4575 of 5004 in epoch 3,loss=1.884, acc=0.543 (749.130 img/s).
Finished step 4576 of 5004 in epoch 3,loss=2.016, acc=0.500 (733.542 img/s).
Finished step 4577 of 5004 in epoch 3,loss=1.944, acc=0.512 (672.782 img/s).
Finished step 4578 of 5004 in epoch 3,loss=1.677, acc=0.633 (601.439 img/s).
Finished step 4579 of 5004 in epoch 3,loss=1.842, acc=0.543 (709.549 img/s).
Finished step 4580 of 5004 in epoch 3,loss=2.113, acc=0.520 (512.565 img/s).
Finished step 4581 of 5004 in epoch 3,loss=2.072, acc=0.551 (751.725 img/s).
Finished step 4582 of 5004 in epoch 3,loss=2.068, acc=0.512 (572.931 img/s).
Finished step 4583 of 5004 in epoch 3,loss=1.848, acc=0.598 (683.068 img/s).
Finished step 4584 of 5004 in epoch 3,loss=2.046, acc=0.512 (723.887 img/s).
Finished step 4585 of 5004 in epoch 3,loss=2.089, acc=0.492 (810.527 img/s).

Finished step 4680 of 5004 in epoch 3,loss=1.958, acc=0.516 (765.376 img/s).
Finished step 4681 of 5004 in epoch 3,loss=1.869, acc=0.551 (566.304 img/s).
Finished step 4682 of 5004 in epoch 3,loss=1.934, acc=0.551 (711.545 img/s).
Finished step 4683 of 5004 in epoch 3,loss=1.922, acc=0.539 (724.513 img/s).
Finished step 4684 of 5004 in epoch 3,loss=1.989, acc=0.516 (740.169 img/s).
Finished step 4685 of 5004 in epoch 3,loss=1.972, acc=0.520 (680.044 img/s).
Finished step 4686 of 5004 in epoch 3,loss=2.084, acc=0.543 (705.202 img/s).
Finished step 4687 of 5004 in epoch 3,loss=1.900, acc=0.555 (657.904 img/s).
Finished step 4688 of 5004 in epoch 3,loss=1.873, acc=0.547 (697.768 img/s).
Finished step 4689 of 5004 in epoch 3,loss=1.929, acc=0.559 (651.628 img/s).
Finished step 4690 of 5004 in epoch 3,loss=1.960, acc=0.555 (669.285 img/s).
Finished step 4691 of 5004 in epoch 3,loss=1.757, acc=0.598 (592.749 img/s).
Finished step 4692 of 5004 in epoch 3,loss=2.051, acc=0.535 (751.831 img/s).

Finished step 4787 of 5004 in epoch 3,loss=1.975, acc=0.535 (655.482 img/s).
Finished step 4788 of 5004 in epoch 3,loss=1.954, acc=0.547 (550.201 img/s).
Finished step 4789 of 5004 in epoch 3,loss=1.980, acc=0.566 (707.856 img/s).
Finished step 4790 of 5004 in epoch 3,loss=1.926, acc=0.566 (712.838 img/s).
Finished step 4791 of 5004 in epoch 3,loss=2.055, acc=0.539 (610.933 img/s).
Finished step 4792 of 5004 in epoch 3,loss=1.998, acc=0.586 (788.657 img/s).
Finished step 4793 of 5004 in epoch 3,loss=1.865, acc=0.555 (688.607 img/s).
Finished step 4794 of 5004 in epoch 3,loss=1.782, acc=0.590 (719.950 img/s).
Finished step 4795 of 5004 in epoch 3,loss=1.837, acc=0.562 (752.831 img/s).
Finished step 4796 of 5004 in epoch 3,loss=1.996, acc=0.500 (739.757 img/s).
Finished step 4797 of 5004 in epoch 3,loss=1.838, acc=0.598 (720.732 img/s).
Finished step 4798 of 5004 in epoch 3,loss=1.959, acc=0.555 (590.915 img/s).
Finished step 4799 of 5004 in epoch 3,loss=1.810, acc=0.543 (703.720 img/s).

Finished step 4894 of 5004 in epoch 3,loss=2.030, acc=0.539 (609.135 img/s).
Finished step 4895 of 5004 in epoch 3,loss=1.910, acc=0.535 (773.565 img/s).
Finished step 4896 of 5004 in epoch 3,loss=2.081, acc=0.504 (642.008 img/s).
Finished step 4897 of 5004 in epoch 3,loss=1.853, acc=0.547 (704.055 img/s).
Finished step 4898 of 5004 in epoch 3,loss=2.039, acc=0.566 (618.197 img/s).
Finished step 4899 of 5004 in epoch 3,loss=1.859, acc=0.574 (630.897 img/s).
Finished step 4900 of 5004 in epoch 3,loss=2.002, acc=0.543 (714.494 img/s).
Finished step 4901 of 5004 in epoch 3,loss=1.769, acc=0.570 (685.943 img/s).
Finished step 4902 of 5004 in epoch 3,loss=1.869, acc=0.566 (736.086 img/s).
Finished step 4903 of 5004 in epoch 3,loss=1.886, acc=0.539 (697.123 img/s).
Finished step 4904 of 5004 in epoch 3,loss=1.827, acc=0.566 (697.435 img/s).
Finished step 4905 of 5004 in epoch 3,loss=2.036, acc=0.527 (580.221 img/s).
Finished step 4906 of 5004 in epoch 3,loss=1.524, acc=0.629 (613.481 img/s).

Finished step 5001 of 5004 in epoch 3,loss=1.894, acc=0.539 (706.160 img/s).
Finished step 5002 of 5004 in epoch 3,loss=1.829, acc=0.574 (671.413 img/s).
Finished step 5003 of 5004 in epoch 3,loss=1.936, acc=0.543 (738.628 img/s).
Finished step 5004 of 5004 in epoch 3,loss=1.844, acc=0.562 (604.652 img/s).
Validation accuracy after epoch 3: 0.2721.


## HomeWork 
### Plot accuracy and Loss for one epoch 
* The loss and accuracies values  for one epoch were saved to a csv file. 


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
data=pd.read_csv("loss_accuracy_data.csv")
print(data)
fig, axs = plt.subplots(figsize=(10,6), 
                       nrows=2, ncols=1)
data["loss"].plot(kind='line',title='loss',x='iteration',y='loss',ax=axs[0])
data["accuracy"].plot(kind='line',title='accuracy',x='iteration',y='loss',ax=axs[1])

FileNotFoundError: [Errno 2] No such file or directory: 'loss_accuracy_data.csv'

In [1]:
plt.savefig("loss_versus_accuracy.png")

NameError: name 'plt' is not defined

In [1]:
import pandas as pd
import tensorflow as tf 
data=pd.read_csv("loss_accuracy_data.csv")

ModuleNotFoundError: No module named 'nums_from_string'

TypeError: findall() missing 1 required positional argument: 'string'